# 0. Introduction

Example based on: Deep Learning with Python by Francois Chollet:
https://www.manning.com/books/deep-learning-with-python

If you need help using the Keras framework: check out the docs at: https://keras.io/

In [1]:
!pip install tensorflow

In [2]:
!pip install keras

In [3]:
import keras
from keras import models
from keras import layers
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array, load_img


import matplotlib.pyplot as plt

# 1. Load data

The MNIST database of handwritten digit has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.
It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting.

In [4]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


11493376/11490434 [==============================] - 0s 0us/step


# 2. Inspect data

Try to get a feel for the data you are using to train and test your neural network. 

## Training data

- Training data will be used to train our neural network to recognize hand-written digits.
- MNIST provides 60000 labeled training images, each 28x28 pixels

In [5]:
train_images.shape

print(train_images[0].shape)

(28, 28)


In [6]:
train_labels.shape

(60000,)

### Exercise 1
Implement the show_image function and randomly select n different samples from the training set to display.

In [7]:
import pandas as pd
def show_image(images, labels, index):
  df = pd.DataFrame(images[index])
  return df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Wistia')


In [8]:
train_images = train_images / 255
show_image(train_images, train_labels, 0)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,0.070588,0.070588,0.070588,0.494118,0.533333,0.686275,0.101961,0.650980,1.000000,0.968627,0.498039,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117647,0.141176,0.368627,0.603922,0.666667,0.992157,0.992157,0.992157,0.992157,0.992157,0.882353,0.674510,0.992157,0.949020,0.764706,0.250980,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.192157,0.933333,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.984314,0.364706,0.321569,0.321569,0.219608,0.152941,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070588,0.858824,0.992157,0.992157,0.992157,0.992157,0.992157,0.776471,0.713725,0.968627,0.945098,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.313725,0.611765,0.419608,0.992157,0.992157,0.803922,0.043137,0.000000,0.168627,0.603922,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Test data

- Test data will be used to validate how good our network performs on data it has never seen.
- MNIST provides 10000 test images, each 28x28.
- It's important to note that these should never be used in the training cycle. A 'test set' should never contain images the network has already seen during training. (read more: [Model Selection and Train/Validation/Test Sets](https://www.coursera.org/lecture/machine-learning/model-selection-and-train-validation-test-sets-QGKbr) and [How (and why) to create a good validation set](https://www.fast.ai/2017/11/13/validation-sets/))

### Exercise 2

Print the test images and test labels shape below: 

In [9]:
print(test_images.shape)
print(test_labels.shape)


(10000, 28, 28)
(10000,)


In [10]:
show_image(test_images, test_labels, 168)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,13,156,172,32,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,80,254,254,59,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,105,254,254,59,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,142,254,254,59,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,142,254,254,101,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,38,254,254,177,0,0,0,0,0,0,0,0,0,0,0,0


# 3. Data Preparation

Before feeding the data into the network for training, we make sure it is formatted properly.

## Prepare the images

### Exercise 3
Reshape the images to vectors of integers

In [11]:
import numpy as np
def reshape(train_images):
    length, height = train_images[0].shape
    reshaped = np.array
    for image in train_images:
      reshaped.append(image.reshape(length * height, 1))
    return reshaped


In [12]:
train_images_reshaped = train_images.reshape(60000, 28 * 28)
test_images_reshaped = test_images.reshape(10000, 28 * 28)


In [13]:
train_images_reshaped.shape

(60000, 784)

### Exercise 4
Resize the values to a value between 0 and 1

In [15]:
train_images_transformed = train_images_reshaped / 255
test_images_transformed = test_images_reshaped / 255

'''
The None dimension is the batch dimension. In other words, the input should have the shape (batch_size, height, width, num_channels).

If you want to predict on one input, change model(image_batch[0]).numpy() to model(image_batch[0:1]).numpy().
 This will maintain the first dimension. The shape will be (1, 40, 40, 3) in this case.
'''
img = test_images[168] / 255
x = img_to_array(img)
x = np.expand_dims(x, axis=0)

## Prepare the labels

In [16]:
?to_categorical

In [17]:
train_labels_categorical = to_categorical(train_labels)
test_labels_categorical = to_categorical(test_labels)

In [18]:
train_labels_categorical[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

# 3. Network architecture

Define the network architecture that will be used for training

- how many layers 
- which type of layer (we'll see more types later)
- how many nodes in each layer
- activation function in each layer


### Exercise 5

1. Start with the simple Neural Network below
2. Start trying out different hyperparameters
  
  a. Change the 'relu' activation function to 'sigmoid'. What happens? (run the cells below to see a difference in test results)

  b. What happens if you add hidden layers? (try the visual example of the slides with a first hidden layer of 300 neurons, and a second one with 100 neurons)

  c. What happens when you try different initializers
  
  b. Figure out how to add "Dropout" to the first layer of the network.

  d. What happens if you add L1 or L2 regularization to the activations?

=> Start with one regularization or optimization parameter at a time. 

=> Experiment with extreme values

=> Every time you try something new, inspect the training cycle of the model and the accompanying graphs

In [19]:
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))

# The final layer has 10 nodes, each node represents one class of numbers
network.add(layers.Dense(10, activation='softmax'))

### Exercise 5b
Try adding a BatchNorm layer. There is some discussion in the Deep Learning community about when to apply Batch Normalization. Either in between the linear part and the nonlinear part of your neuron, or after the nonlinear part.

Although there is no correct answer, the authors of Batch Normalization say that It should be applied immediately before the non-linearity of the current layer. The reason (quoted from original paper):

> "We add the BN transform immediately before the nonlinearity, by normalizing x = Wu+b. We could have also normalized the layer inputs u, but since u is likely the output of another nonlinearity, the shape of its distribution is likely to change during training, and constraining its first and second moments would not eliminate the covariate shift. In contrast, Wu + b is more likely to have a symmetric, non-sparse distribution, that is “more Gaussian” (Hyv¨arinen & Oja, 2000); normalizing it is likely to produce activations with a stable distribution."

You can add it after the activation in the following way:

`model.add(Dense(64, init='uniform', activation='relu')`

`model.add(BatchNormalization())`

You can add it in between the linear and nonlinear part in the following way:

`model.add(Dense(64, init='uniform'))`

`model.add(BatchNormalization())`

`model.add(Activation('tanh'))`



# 4. Compilation Step

In the compilation step we define:

- the loss function
- the optimizer
- the evaluation metric

### Exercise 6

Experiment with different optimizers (SGD, Momentum, RMSProp, Adam, Nadam, etc).

Do they improve the accuracy of the test set?

Change the optimizer from "RMSProp" to "Adam" with good default values:

- learning rate of 0.01
- beta_1 of 0.9
- beta_2 of 0.999
- no epsilon
- learning rate decay (annealing)


What happens if you change the optimizer to good old SGD? Or to Nadam? 

In [ ]:
network.compile(optimizer='SGD',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

# 6. Network summary

In [ ]:
network.summary()

# 7. Train the network

Feed the training images and labels to the network.

Two additional parameters need to be supplied:

- epochs: how many times the network will look at the entire dataset. 
- batch_size: how many images will be put through the network at one time.

### Exercise 7

- Can you train the network longer?
- What happens when you adjust the batch size?
- What happens to the training accuracy?
- And the test accuracy?
- Try adding an early stopping callback function and check the effect it has on your model

In [ ]:
history = network.fit(train_images_transformed, train_labels_categorical, validation_split=0.33, epochs=20, batch_size=128)

In [ ]:
model.save('./')

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# 8. Test the network

Use the test set (which the network has not seen yet) to test how well the network will perform on images it has not seen yet:

In [ ]:
test_loss, test_acc = network.evaluate(test_images_transformed, test_labels_categorical)

In [ ]:
print('test_acc; ', test_acc)

In [ ]:
X_input = Input((28,28,1))

X = Conv2D(5, (3, 3), strides = (1, 1), name = 'conv0')(X_input)
X = BatchNormalization(axis = 3, name = 'bn0')(X)
X = Activation('relu')(X)
X = MaxPooling2D((2, 2), name='max_pool_0')(X)

X = Conv2D(5, (3,3), strides = (1, 1), name = 'conv1')(X)
X = BatchNormalization(axis = 3, name = 'bn1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((2, 2), name='max_pool_1')(X)


X = Flatten()(X)

X = Dense(845, activation='relu', name='fc0')(X)

X = Dense(125, activation='relu', name='fc1')(X)

X = Dense(10, activation='softmax', name='fc2')(X)

model = Model(inputs = X_input, outputs = X, name='numberModel')


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(train_images, train_labels, batch_size=128, epochs=35)
scores = model.evaluate(test_images, test_labels)

#9. Learning rate finder
### Optional exercise

Challenge: 
Can you find a Learning rate finder in Keras? This is native in Fast AI and will be covered more in depth in the coming weeks.
It is, however, not native to Keras. 
